In [13]:
#############################################
##  Node-level Resource Untilization Plot
##     (for every VTK ouput step)
#############################################
# Note: "Nodename per rank" are listed in a single line following every VTK output line

import numpy as np
import linecache
#%matplotlib inline # suppress interactive display by default
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [26]:
###################################
##### PROVIDE INPUTS HERE !!! #####
###################################

# Specify the input file to read
filename = "/media/data/emily/Desktop/yes_vmc8n_swe_impinodes_vtk_3hrs/console.out"

# Specify the output figure path & name
figname_prefix = '/media/data/emily/Desktop/yes_vmc8n_swe_impinodes_vtk_3hrs/resource_pdf_frames/res_'
figname_suffix = '.pdf'

# Specify application number (this determines the color used the application)
appid = 1

# Setup the Nodes visual grid size: nx-by-ny
nx,ny = (8,4)
# Specify CPUs per node: cx-by-cy
cx,cy = (4,4)


In [27]:
###########################
##### Data Processing #####
###########################

# Function for removing duplicate nodes
# Return a list containing only unique nodes
def get_unique(node_list):
    unique_nodes = []
    for n in node_list:
        if n not in unique_nodes:
            unique_nodes += [n]
    return unique_nodes

# Get unique node lists for every OUTPUT step from file
# The output line looks like:
# "iMPI NODES OUTPUT 21 : 0 0 1 1 2 2"
node_lists = []
output_steps = []

f = open(filename, 'r')
for i, line in enumerate(f,1):
    if "iMPI NODES OUTPUT" in line:
        arr = line.split()
        # Output step number is in arr[3]
        output_steps += [int(arr[3])]
        # Starting from arr[5], convert them into a integer array
        nodes = []
        for i in range(5, len(arr)):
            nodes += [int(arr[i])]
        # Reduce to unique nodes
        unique_nodes = get_unique(nodes)
        # append to node lists
        node_lists += [unique_nodes]
f.close()

### For testing only
for i in range(len(node_lists)):
    print("Step ", output_steps[i], ": ", node_lists[i])


Step  1 :  [0]
Step  1 :  [0]
Step  2 :  [0]
Step  3 :  [0]
Step  4 :  [0]
Step  5 :  [0]
Step  6 :  [0]
Step  7 :  [0]
Step  8 :  [0]
Step  9 :  [0]
Step  10 :  [0]
Step  11 :  [0]
Step  12 :  [0]
Step  13 :  [0]
Step  14 :  [0]
Step  15 :  [0]
Step  16 :  [0]
Step  17 :  [0]
Step  18 :  [0]
Step  19 :  [0]
Step  20 :  [0]
Step  21 :  [0]
Step  22 :  [0]
Step  23 :  [0, 1, 2, 3, 4, 5, 6, 7]
Step  24 :  [0, 1, 2, 3, 4, 5, 6, 7]
Step  25 :  [0, 1, 2, 3, 4, 5, 6, 7]
Step  26 :  [0, 1, 2, 3, 4, 5]
Step  27 :  [0, 1, 2, 3, 4, 5]
Step  28 :  [0, 1, 2, 3, 4, 5]
Step  29 :  [0, 1, 2, 3, 4, 5]
Step  30 :  [0, 1, 2, 3]
Step  31 :  [0, 1, 2, 3]
Step  32 :  [0, 1, 2, 3]
Step  33 :  [0, 1, 2, 3]
Step  34 :  [0, 1, 2, 3]
Step  35 :  [0, 1]
Step  36 :  [0, 1]
Step  37 :  [0, 1]
Step  38 :  [0, 1]
Step  39 :  [0, 1]
Step  40 :  [0, 1]
Step  41 :  [0, 1]
Step  42 :  [0, 1]
Step  43 :  [0, 1]
Step  44 :  [0, 1]
Step  45 :  [0, 1]
Step  46 :  [0, 1]
Step  47 :  [0, 1]
Step  48 :  [0]
Step  49 :  [0]
Ste

In [28]:
################################
##### Plotting Definitions #####
################################

# define color code
colorcode = [
        'grey',      # CPU is idle or in transition state
        'lightpink', # CPU is assigned to App1
        'skyblue',   # CPU is assigned to App2
        'khaki',     # CPU is assigned to App3
        'lightgreen' # CPU is assigned to App4
]

# Node patch config
node_size_x = 2.0
node_size_y = 2.0
node_patch_size_x = node_size_x * 0.9
node_patch_size_y = node_size_y * 0.9
node_patch_offset_x = (node_size_x - node_patch_size_x) * 0.5
node_patch_offset_y = (node_size_y - node_patch_size_y) * 0.5

# CPU patch config
cpu_size_x = node_patch_size_x / float(cx)
cpu_size_y = node_patch_size_y / float(cy)
cpu_patch_size_x = cpu_size_x * 0.9
cpu_patch_size_y = cpu_size_y * 0.9
cpu_patch_offset_x = (cpu_size_x - cpu_patch_size_x) * 0.5
cpu_patch_offset_y = (cpu_size_y - cpu_patch_size_y) * 0.5

# Function to get node color
def get_node_color(n, node_list):
    return colorcode[appid] if (n in node_list) else colorcode[0]

# Get the node patch position (x,y)
def getxy(node_id):
    y = ny - 1 - int(node_id / nx)
    x = int(node_id % nx)
    return (x * node_size_x + node_patch_offset_x, \
            y * node_size_y + node_patch_offset_y)

# Get the cpu patch position
def getxy_cpu(node_id, cpu_x, cpu_y):
    (x,y) = getxy(node_id)
    return (x + cpu_x*cpu_size_x + cpu_patch_offset_x, \
            y + cpu_y*cpu_size_y + cpu_patch_offset_y)

# add cpus patches for a node to...
def add_cpus_patches(cpus, node_id, node_list):
    cpu_color = get_node_color(node_id, node_list)
    # Add list of cpu patches for the node
    for j in range(cy):
        for i in range(cx):      
            cpus[node_id*(cx*cy) + (j*cx+i)] = patches.Rectangle(
                    getxy_cpu(node_id, i,j),
                    cpu_patch_size_x,
                    cpu_patch_size_y,
                    facecolor=cpu_color)

# Function for drawing 1 frame
def draw_frame(frame_id, node_list):
    # Figure setup
    fig = plt.figure()
    ax = fig.add_subplot(111, aspect='equal')
    ax.set_xlim([0,nx*node_size_x])
    ax.set_ylim([0,ny*node_size_y])
    ax.axis('off')
    ax.set_frame_on(False)
    # Create a list cpu patches (cx*cy patches per node) for drawing
    cpus = dict()
    for n in range(nx*ny):
        add_cpus_patches(cpus, n, node_list)
    # Draw the node patches
    for c in cpus:
        ax.add_artist(cpus[c]) 
    # Write node number to each node (fancy)
    for n in range(nx*ny):
        nodepatch = patches.Rectangle(getxy(n), node_patch_size_x, node_patch_size_y)
        x,y = nodepatch.get_xy()
        center_x = x + nodepatch.get_width()/2.0
        center_y = y + nodepatch.get_height()/2.0
        # Put node number at the center of the patch
        ax.annotate(n, (center_x, center_y), \
                    color='black', weight='normal', fontsize=12, ha='center', va='center')    
    # Save and close figure
    filename = figname_prefix + str(frame_id) + figname_suffix
    plt.savefig(filename)
    plt.close(fig)


In [30]:
# Test plotting for 1 frame
frame_id = 15
l = node_lists[frame_id]
print(l)
draw_frame(frame_id, l)

[0]


In [31]:
# Draw all frames
for i in range(len(node_lists)):
    draw_frame(output_steps[i],node_lists[i])
    

In [32]:
print(output_steps)

[1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90]
